# load library

In [1]:
source("r/jupyter_common.R")
source("r/plot_sc_clusters.R")


In [2]:
# data structure
suppressPackageStartupMessages(library(data.table))

# single cell analysis
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(ArchR))


## parallel

In [3]:
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(future))
plan("multicore", workers = 8)

options(future.globals.maxSize = 5 * 1024^3) # 5GB


In [4]:
addArchRThreads(threads=2)
getArchRThreads()


addArchRGenome("hg38")


Setting default number of Parallel threads to 2.



[1] 2

Setting default genome to Hg38.



# parameters

In [5]:
str_condition <- "male-bc"

args <- list()
args$cancer_type <- "male-bc"
args$method_to_identify_cell_types <- "singler_blueprint_encode"
args$harmony_theta <- 0
args$seed_kmeans <- 4

str_column_of_meta_data_harmony <- sprintf("RNA_harmony_th.%s", paste(args$harmony_theta, collapse=","))

str_reduction <- "pca"
str_umap_reduction <- "umap"  
col_cell.type <- "cell.type"

if (args$harmony_theta >= 0) {
    str_column_of_meta_data_cluster <- str_column_of_meta_data_harmony
    str_reduction <- "harmony"
    str_umap_reduction <- "umapharmony"
    col_cell.type <- "cell.type.harmony"
}


str_column_of_meta_data_cluster
str_umap_reduction
col_cell.type


[1] "RNA_harmony_th.0"

[1] "umapharmony"

[1] "cell.type.harmony"

## set.seed

In [6]:
# set seed for reproducibility
set.seed(51)



## display

In [7]:
options(repr.matrix.max.cols=150, repr.matrix.max.rows=50)



In [8]:
addArchRVerbose(verbose = FALSE)


Setting addArchRVerbose = FALSE



# read scATAC-seq

In [9]:
dir_atac <- "/home/hkim77/francolab.w/sc-atac-seq/male-bc/run-20220725"
fname_atac <- sprintf("%s/output_p2g_male-bc/rds/male-bc_archrproj_obj_p2gs.rds", dir_atac)

proj.archr <- readRDS(fname_atac)


## list_sort_atac

In [10]:

list_sort_atac <- sort_cluster_members(proj.archr, args,
                    col_cluster_types = "predictedGroup_ArchR",
                    str_umap_reduction = str_umap_reduction,
                    f_merge_immune_cell = FALSE)



Sample,Endothelial cells,Epi. Tumor,Fibroblasts,Macrophages
<chr>,<int>,<int>,<int>,<int>
446B7L,2204,1372,1622,183
4CC61L,NA,2629,404,686


Sample
<chr>
446B7L
4CC61L


Sample,Endothelial cells,Epi. Tumor,Fibroblasts,Macrophages
<chr>,<int>,<int>,<int>,<int>
446B7L,2204,1372,1622,183
4CC61L,NA,2629,404,686


## palGroup

In [11]:

palGroup <- list_sort_atac$nv_color_cluster_type_ordered
palGroup



0-Epi. Tumor        2-Epi. Tumor       4-Fibroblasts 3-Endothelial cells 
          "#A0E989"           "#245719"           "#FABFD2"           "#93CEFF" 
      7-Macrophages 
          "#EEC900"

In [12]:
names(list_sort_atac$nv_color_cluster_type_ordered)


[1] "0-Epi. Tumor"        "2-Epi. Tumor"        "4-Fibroblasts"      
[4] "3-Endothelial cells" "7-Macrophages"

# read p2g.df.sub.plot

In [13]:
dir_rds <- sprintf("%s/output_p2g_male-bc/rds", dir_atac)
fname_p2g.df.sub.plot <- sprintf("%s/p2g.df.sub.plot_enhancer.rds", dir_rds)

p2g.df.sub.plot <- readRDS(fname_p2g.df.sub.plot)


# heatmap_list

In [14]:
source("./r/archr/ArchR-1.0.1/R/ArchRHeatmap.R")
source("./r/archr/ArchR-1.0.1/R/HiddenUtils.R")
source("./r/archr/ArchR-1.0.1/R/LoggerUtils.R")
source("./r/archr/ArchR-1.0.1/R/ValidationUtils.R")
source("./r/archr/archr_v0.9.5_modified/plotPeak2GeneHeatmap.distal.R")

cat(sprintf("\tplotPeak2GeneHeatmap.distal with seed_kmeans=%g\n", args$seed_kmeans))

dir_log <- "log"
heatmap_list <- plotPeak2GeneHeatmap.distal(proj.archr,
                peaks = p2g.df.sub.plot,
                groupBy = "predictedGroup_ArchR",
                k = length(levels(factor(proj.archr$predictedGroup_ArchR))),
                corCutOff = .45,
                varCutOffATAC = 0,
                varCutOffRNA = 0,
                FDRCutOff = 1,
                palGroup = palGroup,
                returnMatrices = FALSE,
                nPlot = 100000,
                seed = args$seed_kmeans,
                legend_ncol = 3
        ) # plotPeak2GeneHeatmap.distal



	plotPeak2GeneHeatmap.distal with seed_kmeans=4


Adding Annotations..

Preparing Main Heatmap..

Adding Annotations..

Preparing Main Heatmap..



# print

In [15]:
filename_heatmap <- "pdf/heatmap_peak2gene_legend.pdf"
cat(sprintf("\tfilename_heatmap: %s\n", filename_heatmap))

pdf(filename_heatmap, width = 8, height = 10)
draw(heatmap_list, heatmap_legend_side = "bottom", annotation_legend_side = "bottom")
dev.off()



	filename_heatmap: pdf/heatmap_peak2gene_legend.pdf


png 
  2

# session info

In [16]:
writeLines(capture.output(sessionInfo()), "txt/sessionInfo.txt")


# reference

https://satijalab.org/seurat/
